# Hack Zurich - Fringer

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS, Isomap, TSNE
from sklearn.decomposition import PCA

from bokeh.resources import CDN
from bokeh.embed import file_html

from fringiness import *
from bokeh.plotting import show

In [73]:
times      = """
WASHINGTON — President Trump seized on a terrorist attack in London on Friday to promote his ban on travel to the United States by Muslims and, in the process, ran afoul of Britain by claiming without evidence that the assailants had been “in the sights” of the British police.
In a fusillade of early morning tweets, Mr. Trump cited the chaotic scene in a London Underground station as Exhibit A for his hard-line policies. His ban on visitors from predominantly Muslim countries, he wrote, should be “far larger, tougher and more specific” — a statement that seemed calculated to mollify his political base after a week in which Mr. Trump suddenly began playing dealmaker with Democrats on immigration.
But Mr. Trump’s assertion that the assailants had been known to Scotland Yard angered Prime Minister Theresa May, who said it was not helpful for anyone to speculate while an investigation was underway. Mr. Trump was later briefed about the attack and called Mrs. May with condolences, according to a senior official, though he did not apologize.
It was the latest episode in which Mr. Trump was at odds with Britain over sensitive security issues. In June, he criticized London’s mayor, Sadiq Khan, over his response to another terrorist attack, misconstruing Mr. Khan’s words. In March, the White House press secretary at the time, Sean Spicer, repeated allegations that a British intelligence agency had wiretapped Trump Tower, which British authorities dismissed as “utterly ridiculous.”
Trump’s assertions were also a sign that for all the talk about a more disciplined White House under the new chief of staff, John F. Kelly — who has urged the president to have tweets vetted by his aides — Mr. Trump was still determined not to censor himself on social media and was fully capable of roiling the diplomatic waters with a single unguarded post.
At 6:42 a.m., Mr. Trump tweeted that “sick and demented people who were in the sights of Scotland Yard” carried out the attack, which injured at least 29 people in the blast and ensuing panic. It was not clear where Mr. Trump got that information, though 23 minutes earlier, “Fox & Friends,” a program he regularly watches, broadcast a report in which a security analyst said he feared that the London police had already known the identity of the attackers.
“Can someone tell Scotland Yard?” asked Brian Kilmeade, one of the hosts of the program.
White House officials tried to play down the contretemps, saying Mr. Trump was referring to the longstanding efforts of British law enforcement authorities to investigate would-be terrorists, not to anyone involved in Friday’s attack.
“What the president was communicating is that obviously all of our law enforcement efforts are focused on this terrorist threat for years,” said the national security adviser, Lt. Gen. H. R. McMaster. “Scotland Yard has been a leader, as our F.B.I. has been a leader.”
The press secretary, Sarah Huckabee Sanders, said she believed that Mr. Trump’s tweet had come up in the conversation with Mrs. May, though she did not characterize the exchange. Mr. Trump, for his part, referred to Mrs. May as “a wonderful woman.” But Mrs. May made her displeasure clear earlier in the day when she told reporters, “I never think it’s helpful for anybody to speculate on what is an ongoing investigation.”
Others were more scathing. Mrs. May’s former chief of staff, Nick Timothy, wrote on Twitter, “True or not — and I’m sure he doesn’t know — this is so unhelpful from leader of our ally and intelligence partner.”
The president has had a strained relationship with Britain since before he took office. Soon after he was elected, he suggested that Nigel Farage, a leader of Britain’s movement to leave the European Union, should be made the British ambassador to the United States — a decision that, London pointed out, was not Mr. Trump’s to make.
Mrs. May and Mr. Trump have had an awkward relationship, going back to their first meeting, when people criticized the way he gripped her hand. Last month, she joined other world leaders in criticizing his response to the white supremacist violence in Charlottesville, Va.
For Mr. Trump, the news of another terrorist attack also offered a moment to do some political repositioning by returning to themes that have animated him since the presidential campaign. After a week in which his pursuit of a compromise with Democrats on immigration rattled his supporters, the president appeared eager to seize on a range of red-meat issues to prove his bona fides.
In addition to the travel ban, Mr. Trump said that no agreement protecting undocumented immigrants brought to the United States as children could include “chain migration” — shorthand among immigration restrictionists for laws that enable immigrants to petition the government for papers for family members.
“Trump underestimated the reaction of throwing DACA out there for debate, and now he’s telling us he really wanted to use that to get all the things he could get on enforcement,” said Representative Steve King, Republican of Iowa and an immigration hard-liner. “I don’t think he realized the base was going to blow up on him.”
Mr. Trump even weighed in on the dispute over Jemele Hill, a black female commentator at ESPN, who tweeted that the president was a “white supremacist.” “ESPN is paying a really big price for its politics,” he tweeted. “Apologize for untruth!” Ms. Hill has apologized, but Ms. Sanders said her comment was a “fireable offense.”
Some polls have shown that the president’s political base would mostly support preserving Deferred Action for Childhood Arrivals, or DACA, which temporarily shields 800,000 young immigrants from deportation. But Mr. King said he was skeptical of those numbers.
Neil Newhouse, a Republican pollster who conducted one of the surveys, said Mr. Trump was engaging in an exercise that he believes has been effective in the past. “He has an ability to hold up a shiny object and his base follows it unerringly, and he’s really good at it,” Mr. Newhouse said.
Mr. Trump also does not hesitate to point fingers after suspicious explosions in European cities. Some counterterrorism experts were also asking whether Friday’s explosion was another instance in which the police overseas had mishandled tips about a possible attack. On “Fox & Friends” shortly before the president tweeted, Jim Hanson, a security analyst who served in the Army Special Forces, said, “My fear, again, is that we’re going to find out this is someone who is known to police.”
A website for the firm Mr. Hanson runs, Security Studies Group, says it focuses on “defending the value of American power against the true threats we face” against a Washington elite that has been “unable or unwilling to address and communicate the most basic requirements of American nationhood.”
"""

infowars = """“Having cut a deal with Democrats for help with the debt ceiling, will Trump seek a deal with Democrats on amnesty for the ‘Dreamers’ in return for funding for border security?”
The answer to that question, raised in my column a week ago, is in. Last night, President Donald Trump cut a deal with “Chuck and Nancy” for amnesty for 800,000 recipients of the Deferred Action for Childhood Arrivals program who came here illegally as youngsters, in return for Democratic votes for more money for border security.
According to preening Minority Leader Pelosi, the agreement contains not a dime for Trump’s Wall, and the “Dreamers” are to be put on a long glide “path to U.S. citizenship.”
Trump denies this is amnesty, and says the Wall comes later.
Fallout? Among the most enthusiastic of Trump backers, disbelief, disillusionment and wonderment at where we go from here.
Trump’s debt-ceiling deal cut the legs out from under the GOP budget hawks. But amnesty would pull the rug out from under all the folks at those rallies who cheered Trump’s promise to preserve the country they grew up in from this endless Third World invasion.
For make no mistake. If amnesty is granted for the 800,000, that will be but the first wave. “There are reasons no country has a rule that if you sneak in as a minor you’re a citizen,” writes Mickey Kaus, author of “The End of Equality,” in The Washington Post.
“We’d be inviting the world. … (An amnesty) would have a knock-on effect. Under ‘chain migration’ rules established in 1965 … new citizens can bring in their siblings and adult children, who can bring in their siblings and in-laws until whole villages have moved to the United States.
“(T)oday’s 690,000 dreamers would quickly become millions of newcomers who may well be low-skilled and who would almost certainly include the parents who brought them — the ones who in theory are at fault.”
Trump is risking a breach in the dam. If the populists who provided him with decisive margins in Ohio, Wisconsin, Michigan and Pennsylvania feel betrayed, it’s hard to blame them.
Why did Trump do it? Clearly, he relished the cheers he got for the debt ceiling deal and wanted another such victory. And with the rampant accusations of a lack of “compassion” for his cancellation of the temporary Obama administration amnesty, he decided he had had enough heat.
It is not easy to stand up for long to the gale force winds of hostile commentary that blow constantly through this city.
Trump’s capitulation, if that is what turns out to be, calls to mind George H. W. Bush’s decision in 1990 to raise the Reagan tax rates in a deal engineered for him by a White House-Hill coalition, that made a mockery of his “Read my lips! No new taxes!” pledge of 1988.
For agreeing to feed the beast of Big Government, rather than cut its rations as Reagan sought to do, Bush was called a statesman.
By the fall of ’92, the cheering had stopped.
Can Trump not know that those congratulating him for his newfound flexibility will be rejoicing, should Bob Mueller indict his family and his friends, and recommend his impeachment down the road?
What makes pre-emptive amnesty particularly disheartening is that the Trump policy of securing the border and returning illegal immigrants to their home countries appears, from a Census Bureau report this week, to be precisely the prescription America needs.
In 2016, paychecks for U.S. households reached an average of $59,039, up 3.2 percent from 2015, a year when they had surged.
U.S. median household income is now at its highest ever.
Yet there are inequalities. Where the median family income of Asian-Americans is above $81,400, and more than $65,000 for white Americans, the median family income of Hispanic families is $47,675, and that of African-American households far less, $39,490.
Consider. Though black Americans are predominantly native-born, while high percentages of Hispanics and Asians are immigrants, from the Census numbers, Hispanics earn more and Asians enjoy twice the median family income of blacks, which is below where it was in 2000.
Still, black America remains steadfastly loyal to a party that supports the endless importation of workers who compete directly for jobs with them and their families. Writes Kaus, “The median hourly wage (of DACA recipients) is only $15.34, meaning that many are competing with hard-pressed, lower-skilled Americans.”
Looking closer at the Census Bureau figures, Trumpian economic nationalism would appear to have its greatest appeal to the American working class, a huge slice of which is native-born, black and Hispanic.
The elements of that policy?
Secure the border. Halt the invasion of low-wage workers, here legally and illegally, from the Third World. Tighten the labor market to force employers to raise wages in our full-employment economy. Provide tax incentives to companies who site factories in the USA. Impose border taxes on the products of companies who move plants abroad.
Put America and American workers first.
Will any amnesty of undocumented workers do that?
"""

fox = """
North Korean strongman Kim Jong Un said his country is close to achieving military "equilibrium" with the United States as a way to make American leaders "dare not talk" about military options for dealing with Pyongyang.
Kim’s comments Saturday came a day after the rogue country fired its latest missile over Japan, a test the United Nations Security Council strongly condemned as “highly provocative.” 
The U.N. slapped North Korea with new sanctions just days ago in response to a powerful Sept. 3 nuclear test.
"""

flat_earth = """
About 2,000 years ago, when science was nowhere near as advanced as it is today, when people hadn’t crossed the seemingly endless oceans separating the continents and observation of Earth from an external vantage point was impossible, most civilizations around the world believed the planet to be a flat surface, often depicted as a plane covered by a circular dome of the sky.
This conception, which held Earth to be a flat surface, is called the flat Earth theory. And while it was already being disputed, based on empirical evidence, as long ago as the 6th century BCE, in some places such as ancient Greece, it persisted in different parts of the world till much more recently, such as in China, where the belief was prevalent till the 17th century.
While it is a misconception that the belief was widespread in Europe during the Middle Ages (the Earth had already been circumnavigated by 1521), there still exist societies to this day that promote the illogical belief, which is contrary to all evidence, including actual photographs of the planet taken from space.
Camilla the rubber chicken is seen at 124,800 feet (38,039 m) above California in this NASA handout image dated March 3, 2012, which also clearly shows the curved surface of Earth. Photo: REUTERS/NASA/Earth to Sky/Bishop Union High School/Handout
The most prominent of the modern groups that believe the Earth to be flat is the Flat Earth Society, which claims on its website: “This is the home of the world-famous Flat Earth Society, a place for free thinkers and the intellectual exchange of ideas. This website hosts information and serves as an archive for Flat Earth Theory. It also offers an opportunity to discuss this with the Flat Earth community on our forums.”
Earlier this year, the rapper known as B.o.B. seemed to endorse the theory as well when he went on a Twitter rant.
Anthony Scaramucci, a Wall Street investor and an adviser to President-elect Donald Trump, in an interview Wednesday likened the science behind climate change to the flat Earth theory, saying scientists didn’t always get things right.
"""


res_infowars = fastrun(infowars)
print(len(res_infowars['environs']))
res_times = fastrun(times)
print(len(res_times['environs']))
res_fox = fastrun(fox)
print(len(res_fox['environs']))
res_flat_earth = fastrun(flat_earth)
print(len(res_flat_earth['environs']))

63
89
53
35
getting entities tag:reuters.com,2017:newsml_KCN1BQ2IJ:2
getting entities tag:reuters.com,2017:newsml_KCN1BQ2HZ:2
35


In [85]:
x, y, f = fringiness(res_to_matrix(res_times)[0], 'correlation')
plot = embedding_plot_bokeh(x, y, f, res_times)
histogram = histogram_bokeh(f)
show(plot)
show(histogram)

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 21), ('ents', 65), ('f', 21), ('numEnts', 65), ('tit', 65), ('x', 21), ('y', 21)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 21), ('ents', 65), ('f', 21), ('fill_color', 21), ('numEnts', 65), ('tit', 65), ('x', 21), ('y', 21)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning

In [86]:
x, y, f = fringiness(res_to_matrix(res_fox)[0])
plot = embedding_plot_bokeh(x, y, f, res_fox)
histogram = histogram_bokeh(f)
show(plot)
show(histogram)

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 10), ('ents', 42), ('f', 10), ('numEnts', 42), ('tit', 42), ('x', 10), ('y', 10)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 10), ('ents', 42), ('f', 10), ('fill_color', 10), ('numEnts', 42), ('tit', 42), ('x', 10), ('y', 10)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning

In [87]:
x, y, f = fringiness(res_to_matrix(res_infowars)[0])
plot = embedding_plot_bokeh(x, y, f, res_infowars)
histogram = histogram_bokeh(f)
show(plot)
show(histogram)

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 17), ('ents', 48), ('f', 17), ('numEnts', 48), ('tit', 48), ('x', 17), ('y', 17)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 17), ('ents', 48), ('f', 17), ('fill_color', 17), ('numEnts', 48), ('tit', 48), ('x', 17), ('y', 17)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning

In [88]:
x, y, f = fringiness(res_to_matrix(res_jacobin)[0])
plot = embedding_plot_bokeh(x, y, f, res_jacobin)
histogram = histogram_bokeh(f)
show(plot)
show(histogram)

/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 6), ('ents', 29), ('f', 6), ('numEnts', 29), ('tit', 29), ('x', 6), ('y', 6)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/usr/local/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python3.6/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('dists', 6), ('ents', 29), ('f', 6), ('fill_color', 6), ('numEnts', 29), ('tit', 29), ('x', 6), ('y', 6)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [ ]:
from bokeh.resources import Resources
rscs = Resources(mode = "relative", root_dir = "/rsc", minified = False)
html = file_html(plot, Resources(), title = "my plot")
with open("file.html", "w") as file:
    file.write(html)

In [ ]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

plot2 = histogram_bokeh(f)

tab1 = Panel(child=plot1, title="Scat")

tab2 = Panel(child=plot2, title="Hist")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)